In [2]:
from sqlalchemy import Column, ForeignKey, Integer, String
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship


In [3]:
Base = declarative_base()

In [4]:
class Patient(Base):
    __tablename__ = "Patients"
    id = Column(Integer, primary_key=True)

    PatientID = Column(String)
    PatientBirthDate = Column(String)
    PatientAge = Column(Integer)
    PatientSex = Column(String)

    def __repr__(self):
        return f"<Patient(PatientID={self.PatientID})>"

In [5]:
import argparse
import sys
import pathlib
import pydicom
import re

In [6]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

In [21]:
def populate_object_from_dicom(object, dcm_obj, tags, extra_fields=None, **kwargs):
    # TODO: Get proper datatype, now everything is a string
    if extra_fields is None:
        extra_fields = {}

    from_dicom = {}
    # This parsing must be more elaborate, checking datatype
    for k in tags[object.__name__]:
        dcm_val = getattr(dcm_obj, k, "NULL")
        if isinstance(dcm_val, pydicom.multival.MultiValue):
            dcm_val = "/".join(dcm_val)
        from_dicom[k] = dcm_val

    return object(**{**from_dicom, **extra_fields}, **kwargs)

In [23]:
def add_or_update(session, model, instance, **kwargs):
    exists = session.query(model).filter_by(**kwargs).first()
    if not exists:
        session.add(instance)
        session.commit()
        return instance

    else:  # TODO: Update only the fields which are new, DICOM does not require each slice to have all the data.
        pass

    return exists


In [24]:
from pydicom.data import get_testdata_file
fpath = get_testdata_file("CT_small.dcm")
from pydicom import dcmread
ds = dcmread(fpath)

In [28]:
 patient = populate_object_from_dicom(Patient, ds, ds.PatientID)

TypeError: string indices must be integers, not 'str'

In [11]:
def import_into_db(session, tags, dcm_fn):
    try:
        dcm_obj = pydicom.read_file(dcm_fn, stop_before_pixels=True)
    except InvalidDicomError:
        return False
    # Create Patient
    # TODO: Create a partial always populating tags
    patient = populate_object_from_dicom(Patient, dcm_obj, tags)
    patient = add_or_update(session, Patient, patient, PatientID=patient.PatientID)

    study = populate_object_from_dicom(Study, dcm_obj, tags, patient=patient)
    study = add_or_update(
        session, Study, study, StudyInstanceUID=study.StudyInstanceUID
    )

    series = populate_object_from_dicom(Series, dcm_obj, tags, study=study)
    series = add_or_update(
        session, Series, series, SeriesInstanceUID=series.SeriesInstanceUID
    )

    image = populate_object_from_dicom(
        Image, dcm_obj, tags, extra_fields={"filename": str(dcm_fn)}, series=series
    )
    # Already imported images are skipped.
    session.add(image)
    session.commit()

    # Check SOPClassUID if this is a MRI.
    if dcm_obj.SOPClassUID == "1.2.840.10008.5.1.4.1.1.4":  # MR Image Storage
        mri_image = populate_object_from_dicom(MRIImage, dcm_obj, tags, image=image)
        session.add(mri_image)
        session.commit()
    return True

In [13]:
def main():
    """Console script for dicomtosql."""
    parser = argparse.ArgumentParser(
        usage="""dicomtosql parses folders recursively for dicom files,
        and saves relevant data to an SQLite database."""
    )
    parser.add_argument(
        "PATH_TO_DICOM", type=pathlib.Path, help="Path to the dicom files."
    )
    parser.add_argument(
        "--sql-database",
        type=pathlib.Path,
        default="dicomtosql.db",
        help="Path to SQLite database.",
    )
    parser.add_argument(
        "--folders",
        type=pathlib.Path,
        help="Path to text file containing paths line by line to process for dicom files.",
    )

    args = parser.parse_args()
    parse_dicom(args.PATH_TO_DICOM, args.sql_database)


if __name__ == "__main__":
    sys.exit(main())  # pragma: no cover



usage: dicomtosql parses folders recursively for dicom files,
        and saves relevant data to an SQLite database.
ipykernel_launcher.py: error: unrecognized arguments: -f


SystemExit: 2

/home/abishek/anaconda3/envs/medphys/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3441: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


NameError: name 'tags' is not defined

In [29]:
!pwd

/home/abishek/Desktop/MedPhys


In [30]:
!ls

dicomDatabase.ipynb  reading_dicom_file.ipynb
